# Fake News Classifier

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.shape

(20800, 5)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [5]:
(df.isnull().sum()/len(df))*100

id        0.000000
title     2.682692
author    9.408654
text      0.187500
label     0.000000
dtype: float64

In [6]:
df = df.dropna()

In [7]:
df.shape

(18285, 5)

In [8]:
(df.isnull().sum()/len(df))*100

id        0.0
title     0.0
author    0.0
text      0.0
label     0.0
dtype: float64

In [9]:
messages = df.copy()

In [10]:
messages.reset_index(inplace=True)

In [11]:
messages.head()

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [12]:
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [13]:
stemmer = PorterStemmer()

In [14]:
title_corpus = []
for i in range(len(messages['title'])):
    review = re.sub('\W',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [stemmer.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    title_corpus.append(review)

In [15]:
title_corpus[3]

'15 civilian kill singl us airstrik identifi'

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000)
X = cv.fit_transform(title_corpus).toarray()    # You need to pass a list in this as an argument

In [23]:
X.shape

(18285, 5000)

In [24]:
type(X)

numpy.ndarray

In [25]:
X = pd.DataFrame(X, columns=cv.get_feature_names())

In [26]:
type(X)

pandas.core.frame.DataFrame

In [27]:
y = messages['label']

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=0)

In [39]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [42]:
classifier = RandomForestClassifier()
params = {'n_estimators' : [5,10,15,20,25]}
model = GridSearchCV(estimator = classifier,
                     param_grid = params,
                     cv = 5,
                     scoring = 'accuracy',
                     verbose = 1)

In [43]:
model.fit(X_train,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:  3.7min finished


GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'n_estimators': [5, 10, 15, 20, 25]},
             scoring='accuracy', verbose=1)

In [44]:
print("Best Accuracy: ", model.best_score_)
print("Best number of estimators: ", model.best_params_)

Best Accuracy:  0.9294068153134202
Best number of estimators:  {'n_estimators': 20}


In [45]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [47]:
final_classifier = RandomForestClassifier(n_estimators = 20)

In [48]:
final_classifier.fit(X_train,y_train)

RandomForestClassifier(n_estimators=20)

In [49]:
y_pred = final_classifier.predict(X_test)

In [50]:
acc_score = accuracy_score(y_test,y_pred)
acc_score

0.93015625

In [51]:
confusion_matrix(y_test,y_pred)

array([[3249,  355],
       [  92, 2704]], dtype=int64)